In [1]:
import fiona
import pyproj
from shapely.ops import transform
from shapely.geometry import shape, mapping

oldFeatures = []
geoJson = 'E:/OSM/Schutzgebiete/Schongebiete.geojson'
with fiona.open(geoJson) as input:
    oldCrs = input.crs
    for feat in input:
        if feat['geometry'] != None:
            oldFeatures.append(feat)
print(len(oldFeatures))

478


In [2]:
# FIXME: convert to 3D for comparison with the provided shape files?
project = pyproj.Transformer.from_proj(
    pyproj.Proj(init='epsg:4326'), # source coordinate system
    pyproj.Proj(init='epsg:31468')) # destination coordinate system

count = 0
for feat in oldFeatures:    
    g = None
    try:
        oldGeom = shape(feat['geometry'])
        g = transform(project.transform, oldGeom)
    except:
        pass
    if g:
        feat['geometry'] = mapping(g)
        count += 1
print(count)

477


In [3]:
print(input.crs)

{'init': 'epsg:4326'}


In [4]:
# schema = {'geometry': 'LineString','properties': {'boundary': 'str:20'}}
# epsg:31468
# TODO: add relevant properties
schema = {'geometry': 'Polygon'}
with fiona.open('E:/OSM/Schutzgebiete/OSM/osm-shapes.shp', 'w', crs={'init':'epsg:31468'}, driver='ESRI Shapefile',schema=schema) as out:
    for feat in oldFeatures:
        myFeat = {}
        myFeat['geometry'] = feat['geometry']
        out.write(myFeat)